In [ ]:
from mnist_load import MNIST
import numpy as np
import matplotlib.pyplot as plt
from util import *

In [9]:
# 0:discrete / 1:continuous
mode = 0

In [ ]:
mndata = MNIST('../data')
mndata.gz = True
images,labels = mndata.load_training()
test_images,test_labels = mndata.load_testing()
images = np.array(images)

KeyboardInterrupt: 

In [ ]:
#for visualize
"""
for i in range(100):
    p = np.array(images[i], dtype='uint8') 
    p = p.reshape((28, 28))
    #print(p)
    plt.imshow(p, cmap='gray')
    plt.show()
    print("label" + str(i) + ":" + str(labels[i]))
"""

'\nfor i in range(100):\n    p = np.array(images[i], dtype=\'uint8\') \n    p = p.reshape((28, 28))\n    #print(p)\n    plt.imshow(p, cmap=\'gray\')\n    plt.show()\n    print("label" + str(i) + ":" + str(labels[i]))\n'

In [ ]:
# get prior
labels = np.array(labels)
prior = get_prior(labels)

[5 0 4 ... 5 6 8]


In [ ]:
if mode == 0:
    # for train
    discrete_bins = np.zeros((10, 28*28, 32))
    for img in range(len(images)):
        c = labels[img]
        for d in range(28*28):
            #print(int(images[img][d])//8)
            discrete_bins[c, d, int(images[img][d])//8] += 1  #每8個gray level分一個bin
    for c in range(10):
        for d in range(28*28):
            count = 0
            for bin in range(32):
                count += discrete_bins[c][d][bin]
            discrete_bins[c][d][:] /= count   #conditional probability of every bin
    # for test
    error_cnt = 0
    for img in range(len(test_images)):
        probs = np.zeros(10)
        for p in range(10):
            for d in range(28*28):
                # value相乘 = log相加
                probs[p] += np.log(max(1e-4, discrete_bins[p, d, test_images[img][d]//8]))
                # 避免log 0的情況 (1/60000 大概是1e-4)
            probs[p] += np.log(prior[p])

        probs /= np.sum(probs)
        print('Posterior (in log scale):')
        for c in range(10):
            print('{}: {}'.format(c, probs[c]))
        predict=np.argmin(probs)  # np.sum是negative，所以這裡取min
        print('Prediction: {}, Ans: {}'.format(predict, test_labels[img]))
        print()
        if predict != test_labels[img]:
            error_cnt += 1
    
    # Imagination of numbers in Bayesian classifier:
    print_imagination_number(discrete_bins, 16)
    print('Error rate: {:.4f}'.format(error_cnt / len(test_images)))


elif mode == 1:
    continuous_prob = np.zeros((10, 28*28, 256))
    # for train
    epsilon_var = 1000  #實驗結果:越大越好
    
    for c in range(10):
        A = images[labels==c]
        for d in range(28*28):
            mu = np.mean(A[:, d])
            var = get_variance(A[:, d], epsilon_var)  #避免var是0
            for j in range(256):
                continuous_prob[c, d, j] = gaussain_prob(j, mu, var)
    # for test
    error_cnt = 0
    for img in range(len(test_images)):
        probs = np.zeros(10)
        for c in range(10):
            for d in range(28*28):
                probs[c] += np.log(max(1e-30, continuous_prob[c, d, test_images[img][d]]))
                # 避免continuous_prob太小log算不出來(實驗結果:越小越好)
            probs[c] += np.log(prior[c])
        # for normalized
        probs /= np.sum(probs)
        print('Posterior (in log scale):')
        for c in range(10):
            print('{}: {}'.format(c, probs[c]))
        predict=np.argmin(probs)
        print('Prediction: {}, Ans: {}'.format(predict, test_labels[img]))
        print()
        if predict != test_labels[img]:
            error_cnt += 1
    print_imagination_number(continuous_prob, 128)
    print('Error rate: {:.4f}'.format(error_cnt / len(test_images)))
    print()
    
            
            
        
